Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*>

else:
    DATA_PATH = '../data/'

In [11]:
import pandas as pd
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100
import category_encoders as cat
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)        ####### NO RANDOMNESS!!
from scipy import stats

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.tree import DecisionTreeClassifier

In [21]:
df = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
              pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv')).set_index('id')

df_test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv', index_col='id')

In [22]:
target = 'status_group'
X = df.drop(columns=target)
y = df[target]

In [23]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2, random_state=42)

In [24]:
X_train.describe(exclude='number').T.sort_values(by='unique').T

,recorded_by,public_meeting,permit,source_class,management_group,quantity_group,quantity,waterpoint_type_group,quality_group,payment_type,source_type,waterpoint_type,extraction_type_class,payment,water_quality,basin,source,scheme_management,management,extraction_type_group,extraction_type,region,lga,date_recorded,funder,installer,ward,scheme_name,subvillage,wpt_name
count,47520,44831,45081,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,44418,47520,47520,47520,47520,47520,47520,44644,44631,47520,24997,47224,47520
unique,1,2,2,3,5,5,5,6,6,7,7,7,7,7,8,9,10,12,12,13,18,21,125,351,1698,1923,2076,2541,17232,30742
top,GeoData Consultants Ltd,True,True,groundwater,user-group,enough,enough,communal standpipe,good,never pay,shallow well,communal standpipe,gravity,never pay,soft,Lake Victoria,shallow well,VWC,vwc,gravity,gravity,Iringa,Njombe,2011-03-15,Government Of Tanzania,DWE,Igosi,K,Shuleni,none
freq,47520,40743,31028,36627,42018,26538,26538,27615,40633,20318,13540,22785,21340,20318,40633,8205,13540,29462,32455,21340,21340,4254,2001,459,7250,13944,242,543,409,2858


In [25]:
def clean_string(inst):
    if isinstance(inst, str):
        return inst.lower().strip()
    elif isinstance(inst, bool):
        return inst
    else:
        return np.NaN


def wrangle(df):
    df = df.copy()
    
    date_recorded = pd.to_datetime(df['date_recorded'])
    
    df['month_recorded'] = date_recorded.dt.month
    df['year_recorded'] = date_recorded.dt.year
    
    ### recorded_by has only 1 unique, so no information
    ### subvillage and wpt_name are VERY high cardinality
    ### permit is least related to the target, as determined by chi2 tests
    
    df = df.drop(columns=['recorded_by', 'subvillage', 'wpt_name',
                          'permit'
                         ])
        
    for feat in df.select_dtypes('object').columns:
        df[feat] = df[feat].apply(clean_string)
            
    return df

In [26]:
X_train = wrangle(X_train)

In [18]:
for feat in X_train.select_dtypes('object').columns:
    print(feat, stats.chi2_contingency(pd.crosstab(X_train[feat], y_train))[1])

date_recorded 0.0
funder 0.0
installer 0.0
basin 0.0
region 0.0
lga 0.0
ward 0.0
public_meeting 1.3313329323489593e-55
scheme_management 2.703367503003533e-307
scheme_name 0.0
permit 1.8570956936315765e-10
extraction_type 0.0
extraction_type_group 0.0
extraction_type_class 0.0
management 0.0
management_group 8.439235574308659e-44
payment 0.0
payment_type 0.0
water_quality 0.0
quality_group 0.0
quantity 0.0
quantity_group 0.0
source 0.0
source_type 0.0
source_class 3.8498650886182996e-97
waterpoint_type 0.0
waterpoint_type_group 0.0


In [27]:
### Split features into categories, to use in our Great Transformer

num_feats = X_train.select_dtypes('number').columns
cat_feats = X_train.select_dtypes('object').columns
cat_feats_low_card = [f for f in cat_feats if X_train[f].nunique() < 100]
cat_feats_high_card = [f for f in cat_feats if X_train[f].nunique() >= 100]

In [28]:
### Used in the Great Transformer. Sets NaNs to the NaN count in that column.

def CountImputer(df):
    for col in range(df.shape[1]):
        df[:, col] = np.nan_to_num(df[:, col], nan=np.count_nonzero(np.isnan(df[:, col])))
    return df

In [11]:
### THE GREAT TRANSFORMER

### This is a tree of its own sort, using FeatureUnions, Pipelines, 
### and ColumnTransformers to split the set by certain categories, 
### perform different encodings, and re-merge.

transformers = FeatureUnion([ 
    
    ##### PCA Section
    ('pca_part', Pipeline([  
        ('encoders_for_pca', ColumnTransformer([           ## Count and OneHot encoders
            
            ('low_card', FeatureUnion([
                ('onehotencoder', cat.OneHotEncoder(use_cat_names=True)),
                ('countencoder', cat.CountEncoder(min_group_size=.01))
            ]), cat_feats_low_card),
             
            ('high_card', cat.CountEncoder(min_group_size=.01), cat_feats_high_card)
            
        ], verbose=True)),
        
        ('realimputer', FunctionTransformer(CountImputer)),      ## Custom imputer defined above.
        ('standardscaler', StandardScaler()),
        ('pca', PCA(n_components=1, random_state=42, svd_solver='arpack'))    ## Only keeping 1 component!
    ])),
    
    
    ##### Normal Section
    ('ordinal_pipe', ColumnTransformer([                                           
         ('ordinalencoder', cat.OrdinalEncoder(), cat_feats),      ## Ordinal encode all cat feats
         ('dumb', FunctionTransformer(lambda df: df), num_feats)   ## Pass all num feats
     ]))
    
])

In [12]:
transformers.fit(X_train)

c:\users\benjs\opencv\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


[ColumnTransformer] ...... (1 of 2) Processing low_card, total=   3.7s
[ColumnTransformer] ..... (2 of 2) Processing high_card, total=   0.1s


FeatureUnion(transformer_list=[('pca_part',
                                Pipeline(steps=[('encoders_for_pca',
                                                 ColumnTransformer(transformers=[('low_card',
                                                                                  FeatureUnion(transformer_list=[('onehotencoder',
                                                                                                                  OneHotEncoder(use_cat_names=True)),
                                                                                                                 ('countencoder',
                                                                                                                  CountEncoder(combine_min_nan_groups=True,
                                                                                                                               min_group_size=0.01))]),
                                                                          

In [13]:
XT_train = pd.DataFrame(transformers.transform(X_train), index=X_train.index)
XT_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
454,-3.525566,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,2092.0,35.426020,-4.227446,0.0,21.0,1.0,160.0,1998.0,2.0,2013.0
510,4.125788,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,35.510074,-5.724555,0.0,1.0,6.0,0.0,0.0,3.0,2011.0
14146,3.747038,3.0,3.0,3.0,2.0,3.0,3.0,3.0,1.0,2.0,1.0,3.0,3.0,3.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,3.0,3.0,0.0,0.0,32.499866,-9.081222,0.0,12.0,6.0,0.0,0.0,7.0,2011.0
47410,-4.919524,4.0,4.0,4.0,3.0,3.0,4.0,4.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,3.0,3.0,2.0,1.0,1.0,0.0,0.0,34.060484,-8.830208,0.0,12.0,7.0,0.0,0.0,4.0,2011.0
1288,5.602780,5.0,5.0,5.0,4.0,4.0,5.0,5.0,1.0,2.0,1.0,3.0,3.0,3.0,2.0,1.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,3.0,300.0,1023.0,37.032690,-6.040787,0.0,5.0,1.0,120.0,1997.0,4.0,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68525,-5.228332,194.0,43.0,6.0,3.0,4.0,57.0,354.0,1.0,1.0,179.0,1.0,1.0,1.0,3.0,1.0,3.0,3.0,1.0,1.0,2.0,2.0,3.0,3.0,2.0,1.0,1.0,1000.0,327.0,36.367112,-8.774761,0.0,5.0,4.0,255.0,1995.0,3.0,2011.0
11980,-4.515904,158.0,11.0,28.0,8.0,9.0,35.0,625.0,1.0,2.0,13.0,1.0,1.0,1.0,2.0,1.0,6.0,6.0,1.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1000.0,1743.0,34.531524,-9.769604,0.0,11.0,5.0,35.0,1988.0,3.0,2011.0
35778,4.363706,24.0,1699.0,1742.0,7.0,8.0,95.0,688.0,2.0,9.0,1.0,3.0,3.0,3.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,3.0,3.0,0.0,-13.0,38.974416,-5.420823,0.0,4.0,5.0,1000.0,2005.0,3.0,2011.0


In [14]:
X_val = wrangle(X_val)
XT_val = pd.DataFrame(transformers.transform(X_val), index=X_val.index)

In [15]:
model = RandomForestClassifier(n_estimators=610,
                               random_state=42, 
                               n_jobs=-1, 
                               max_depth=40,
                               max_leaf_nodes=4500,
                               min_samples_leaf=2,
                               min_samples_split=3)

In [16]:
model.fit(XT_train, y_train)

RandomForestClassifier(max_depth=40, max_leaf_nodes=4500, min_samples_leaf=2,
                       min_samples_split=3, n_estimators=610, n_jobs=-1,
                       random_state=42)

In [17]:
model.score(XT_val, y_val)

0.8154040404040404

In [18]:
# XT_train.to_csv('tt_great.csv')

In [21]:
X_test = wrangle(df_test)

In [22]:
XT_test = pd.DataFrame(transformers.transform(X_test), index=X_test.index)

In [16]:
predictions = pd.DataFrame({'status_group': model.predict(XT_test)}, index=XT_test.index)
predictions.to_csv('pump_submission_great_last_try.csv')